In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")
%matplotlib inline

opinions_df = pd.read_csv('dataset2300.csv', encoding='CP949')
opinions_df.head(3)

,content,star
0,물건 받고 10일만에 액정 나가네요. 처음부터 하자 있는 물건이었다고 저는 생각합니...,0
1,키보드 자판위에 칠이 조금 벗겨져 있어서 받고 기분이 좋지는 않았네요...,0
2,무선마우스 작동안되는거줌 중국산 3천원어치인듯,0


In [ ]:
# E-mail, URL이 포함된 리뷰 제거


In [89]:
from sklearn.model_selection import train_test_split

# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수
def get_train_test_dataset(df=None):
    # 인자로 입력된 DataFrame 복사
    df_copy = df.copy()
    # DataFrame의 맨 마지막 칼럼이 레이블, 나머지는 피처들
    X_features = df_copy.iloc[:,0]
    y_target = df_copy.iloc[:,1]
    # train_test_split()으로 학습과 테스트 데이터 분할. stratify=y_target으로 Stratified기반 분할
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3, random_state=0, stratify=y_target)
    # 학습과 테스트 데이터 세트 반환
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_dataset(opinions_df)

In [90]:
print('학습 데이터 레이블 값 비율')
print(y_train.value_counts()/y_train.shape[0] * 100)
print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0] * 100)

학습 데이터 레이블 값 비율
0    50.182704
1    49.817296
Name: star, dtype: float64
테스트 데이터 레이블 값 비율
0    50.142045
1    49.857955
Name: star, dtype: float64


In [91]:
train_set = pd.concat([X_train, y_train], axis =1)
test_set = pd.concat([X_test, y_test], axis =1 )
train_set = train_set.reset_index()
test_set= test_set.reset_index()

In [92]:
train_set

,index,content,star
0,1852,감사합니다,1
1,2283,판매자분이 너무 친철하십니다!,1
2,1538,그램 15,1
3,883,안주느니만 못하네요 usb연결도 하고 건전지도 넣어야하고근데 디자인도 예뻐서 이해햇...,0
4,1977,부자되세요!,1
...,...,...,...
1637,990,"제품 광고 사진에는 게이밍 헤드셋을 준다고 해서 구매했는데, 리뷰를 작성하고 평가...",0
1638,1142,"하루종일 힘들게 윈도우, 드라이버 깔았더니 그래픽 카드(엔비디아 mx150) 인식을...",0
1639,725,윈도우도 정상으로 설치했고 문제 없다고 생각하고 사용하고 있었습니다. 그리고 구매당...,0
1640,2306,빠른 배송에 감사합니다. 최저가로 해주심에도 감사합니다. 맥프로에 감탄하고 있습니다...,1


# 최빈 단어 추출

In [94]:
from konlpy.tag import Okt

okt = Okt()

s = '이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요~^^'
okt.pos(s)

[('이', 'Noun'),
 ('밤', 'Noun'),
 ('그날', 'Noun'),
 ('의', 'Josa'),
 ('반딧불', 'Noun'),
 ('을', 'Josa'),
 ('당신', 'Noun'),
 ('의', 'Josa'),
 ('창', 'Noun'),
 ('가까이', 'Noun'),
 ('보낼게요', 'Verb'),
 ('~^^', 'Punctuation')]

In [95]:
import re

def tokenize(doc):
    # 한글 자음, 모음 제거
    doc = re.sub(pattern='([ㄱ-ㅎㅏ-ㅣ]+)', repl='', string=doc)
    # 특수기호 제거
    doc = re.sub(pattern='[^\w\s]', repl='', string=doc)
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    doc = okt.pos(doc, norm=True, stem=True)
    # 2글자 이상만 포함
    token = []
    for i in doc:
        if len(i[0])>1:
            token.append(i)
    
    return ['/'.join(t) for t in token]

In [96]:
train_docs = [(tokenize(row['content']), row['star']) for idx, row in tqdm(train_set.iterrows())]
test_docs = [(tokenize(row['content']), row['star']) for idx, row in tqdm(test_set.iterrows())]

# 위에서 만든 데아터에서 긍/부정을 제외하고 token에 넣어준다.
# [[a],b] 에서 a만 넣는다고 생각하면 됨
tokens = [t for d in train_docs for t in d[0]]
tokens

1642it [01:10, 23.21it/s]
704it [00:36, 19.07it/s]


['감사하다/Verb',
 '판매/Noun',
 '너무/Adverb',
 '친철/Noun',
 '하다/Verb',
 '그램/Noun',
 '15/Number',
 '알다/Verb',
 '하다/Verb',
 'usb/Alpha',
 '연결/Noun',
 '하다/Verb',
 '건전지/Noun',
 '넣다/Verb',
 '근데/Adverb',
 '디자인/Noun',
 '예쁘다/Adjective',
 '이해/Noun',
 '햇습니/Noun',
 '근데/Adverb',
 '오른쪽/Noun',
 '살짝/Noun',
 '누르다/Verb',
 '되다/Verb',
 '왼쪽/Noun',
 '엄청/Adverb',
 '엄청/Adverb',
 '엄청/Adverb',
 '엄청/Adverb',
 '엄청/Adverb',
 '세다/Verb',
 '누르다/Verb',
 '클릭/Noun',
 '되다/Verb',
 '지금/Noun',
 '잠깐/Noun',
 '삼사십분/Noun',
 '쓰다/Verb',
 '스트레스/Noun',
 '이쁘다/Adjective',
 '불량/Noun',
 '같다/Adjective',
 '사은/Noun',
 '받다/Verb',
 '교환/Noun',
 '해달라다/Verb',
 '기도/Noun',
 '그렇다/Adjective',
 '고별/Noun',
 '기분/Noun',
 '좋다/Adjective',
 '마우스/Noun',
 '주문/Noun',
 '하고/Josa',
 '오다/Verb',
 '바로/Noun',
 '버리다/Verb',
 '사은/Noun',
 '이렇다/Adjective',
 '줄다/Verb',
 '그냥/Noun',
 '애초/Noun',
 '주다/Verb',
 '좋다/Adjective',
 '같다/Adjective',
 '기본/Noun',
 '리뷰/Noun',
 '제품/Noun',
 '받다/Verb',
 '기분/Noun',
 '좋다/Adjective',
 '써다/Verb',
 '마우스/Noun',
 '연결하다/Adjective',
 '쓰다/Verb',
 '어이/No

In [97]:
import nltk
#nltk라이브러리를 통해서 텍스트 데이터 나열
text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))            

# 출현 빈도가 높은 상위 토큰 10개
print(text.vocab().most_common(10))

50413
5212
[('하다/Verb', 3227), ('좋다/Adjective', 833), ('배송/Noun', 733), ('노트북/Noun', 714), ('있다/Adjective', 649), ('받다/Verb', 602), ('으로/Josa', 579), ('제품/Noun', 517), ('구매/Noun', 472), ('되다/Verb', 412)]


In [98]:
#단어 빈도수가 높은 10000개의 단어만 사용하여 각 리뷰 문장마다의 평가지표로 삼는다.
selected_words = [f[0] for f in text.vocab().most_common(10000)]

#term_frequency()함수는 위에서 만든 selected_words의 갯수에 따라서 각 리뷰와 매칭하여 상위 텍스트가 
#각 리뷰에 얼만큼 표현되는지 빈도를 만들기 위한 함수
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

# 예측 모델 구축

In [99]:
#모델링을 하기 위해 리스트로 되어 있는 데이터 형식을 array로 바꿔주고 dtype도 실수로 바꿔준다.
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [100]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

#tensorflow.keras를 활용하여 모델의 층 입력하기
model = models.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#모델 생성
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

In [101]:
#모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

#예측 결과
results

Epoch 1/10
4/4 [==============================] - 0s 36ms/step - loss: 0.6642 - binary_accuracy: 0.6657
Epoch 2/10
4/4 [==============================] - 0s 47ms/step - loss: 0.5180 - binary_accuracy: 0.8861
Epoch 3/10
4/4 [==============================] - 0s 37ms/step - loss: 0.4005 - binary_accuracy: 0.9166
Epoch 4/10
4/4 [==============================] - 0s 25ms/step - loss: 0.3213 - binary_accuracy: 0.9336
Epoch 5/10
4/4 [==============================] - 0s 36ms/step - loss: 0.2665 - binary_accuracy: 0.9482
Epoch 6/10
4/4 [==============================] - 0s 30ms/step - loss: 0.2240 - binary_accuracy: 0.9537
Epoch 7/10
4/4 [==============================] - 0s 27ms/step - loss: 0.1917 - binary_accuracy: 0.9604
Epoch 8/10
4/4 [==============================] - 0s 27ms/step - loss: 0.1658 - binary_accuracy: 0.9683
Epoch 9/10
4/4 [==============================] - 1s 260ms/step - loss: 0.1454 - binary_accuracy: 0.9720
Epoch 10/10
22/22 [==============================] - 0s 4ms/ste

[0.29904788732528687, 0.890625]

In [102]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;\n".format(review, (1 - score) * 100))

In [103]:
predict_pos_neg("2015년식 삼성노트북을 교체했습니다 그래픽카드와 화면크기가 아쉬웠는데 아주만족합니다 터치스크린은 최고내요")
predict_pos_neg("리뷰보고 괜찮아보여서 샀는데 엄청 느리네요. 게임하면 버벅 거리고 돈 더 쓸 걸 그랬어요..")
predict_pos_neg("16인치 소식으로 너무 아쉽게됐지만 잘 쓰고 있어요")

[2015년식 삼성노트북을 교체했습니다 그래픽카드와 화면크기가 아쉬웠는데 아주만족합니다 터치스크린은 최고내요]는 79.64% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[리뷰보고 괜찮아보여서 샀는데 엄청 느리네요. 게임하면 버벅 거리고 돈 더 쓸 걸 그랬어요..]는 90.17% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[16인치 소식으로 너무 아쉽게됐지만 잘 쓰고 있어요]는 87.63% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

